In [ ]:
###########
# Read Me #
###########

# Compile the first cell below to import libraries.
# The second cell below is the code of the simulation.
# The other cells, except for the last and the second last, are the codes of single steps of the simulation.
# Use the code from the second last cell to perform and save simulations for different combinations of parameters.
# Use the code from the last cell to obtain a graphical representetion in the k-p plane. 

# Parameters:
    # M: number of couples;
    # N: number of agents, N=M*2;
    # k: probability of intuitive response;
    # p: probability of one-shot prisoner's dilemma;
    # A: probability of assortativity in cognition for each couple;
    # T: Number of time steps in a simulation round;
    # vector_agents: vector of integer numbers, ranging from 0 to N-1, N=M*2. Each agent is represented by a number;  

In [ ]:
####################
# Import libraries #
####################

%matplotlib notebook
import matplotlib.pyplot as plt
import itertools
import numpy as np
import sympy as sym 
import random as rd

In [ ]:
##############
# Simulation #
##############

def simulation(T,p,k,A,M):
    
    N=M*2
    
    vector_agents= [i for i in range(N)]
    
    memory = np.ones([N,2])                     # Initial memory of agents: both memories of cooperation and defection are equal to one;
    
    total_cooperation_s1=[]                     # Generate empty set. In each period the new value is added. At the end of the simulation a complete time series is generated;         
    total_cooperation_tot=[]                                     
    total_reward_t=[]  
    reward_s1_t=[]     
    
    for t in range(T):                                                                  # In each period:
            
        Matching=matching(M,vector_agents)                                                  # Match agents in random couples
            
        Random_game=random_game(p,M)                                                        # Select the game of each couple
            
        Choice_s1=choice_s1(M,memory)                                                       # Select the intuitive response of each agent
            
        Choice_s2=choice_s2(M,Random_game,Matching)                                         # Select the deliberative response of each agent   
        
        System=system_choice(M,A,k,Matching)                                                # Select the cognitive system of each agent
            
        Choice=choice(M,System,Choice_s1,Choice_s2)                                         # Select the strategy of each agent, given the choice of cognitive system    
            
        Reward=reward(M,Random_game,Choice,Matching)                                        # Calculate the reward of each agents    
        
        Cooperation_s1=cooperation(M,Choice,memory)                                         # Calculate cooperation under intuition    
        
        Cooperation_tot=cooperation_tot(M,Choice)                                           # Calculate total cooperation   

        total_cooperation_s1.append(Cooperation_s1/(N))                                     # Calculate cooperation rate under intuition      
            
        total_cooperation_tot.append(Cooperation_tot/(N))                                   # Calculate total cooperation rate        
            
        total_reward=sum(Reward)/(M)                                                        # Calculate average reward            
        
        total_reward_t.append((total_reward[0]+total_reward[1])/2)                          # Generate a time serie of average total reward

        Reward_s1=reward_intuition(M,System,Reward,Matching)                                # Calculate average reward under intuition
        
        reward_s1_t.append(Reward_s1)                                                       # Generate a time serie of average reward under intuition
        
        ##################################################### 
        # Select one of the two following update strategies #
        #####################################################################################
        memory=update_reinforce(M,Choice,memory,Reward,vector_agents)                       # Agents update memories: reinforcement learning
        #memory=update_memory_1(M,Choice,memory,Reward,vector_agents)                        # Agents update memories: last reward    
        #####################################################################################
        
    total_reward_T=sum(total_reward_t)/T                                                # Calculate the average reward along the entire simulation

    reward_s1_T=sum(reward_s1_t)/T                                                      # Calculate the average reward under intuition along the entire simulation

    mean_coop_s1=sum(total_cooperation_s1)/T                                            # Calculate the average cooperation rate under intuition along the entire simulation 

    mean_coop_tot=sum(total_cooperation_tot)/T                                          # Calculate the average total cooperation rate along the entire simulation   

    return(mean_coop_s1, mean_coop_tot)                                                 # Return the average cooperation rate under intuition and the average total cooperation rate
                                                                                        # Other results can be added, such as the mean average reward under intuition;

In [ ]:
###################
# Random Matching #
###################

def matching(M,vector_agents):
    N=M*2
    couples=np.ones([M,2])
    rd.seed()
    rd.shuffle(vector_agents)                                           # Randomize the order of agents
    for i in np.arange(0,M,1):
        couples[i]=[int(vector_agents[i]),int(vector_agents[N-1-i])]    # Match the i-first and the i-last
    return(couples)

In [ ]:
#############################
# Choice of the game played #
#############################

def random_game(p,M):         # game=0: one-shot prisoner's dilemma; game=1 repeated prisoner's dilemma
    N=M*2
    game=np.zeros(M)
    for i in range(M):              # Loop inside couples
        if p<=rd.uniform(0, 1):         # With probability 1-p
            game[i]=0                       # Couple i plays the one-shot prisoner's dilemma
        else:                           # With probability p
            game[i]=1                       # Couple i plays the repeated prisoner's dilemma
    return(game)

In [ ]:
################################################
# Payoff matrix of one-shot prisoner's dilemma # 
################################################

#     C    D
#  C 4,4  0,5
#  D 5,0  1,1

# action0: action of row player; 0 for cooperation, C; 1 for defection, D;
# action1: action of column player; 0 for cooperation, C; 1 for defection, D;

payoff_matrix_Game_0=np.zeros(2)
def Game_0(action0,action1):
    if action0==0 and action1==0:          # Both the agents cooperate;
        payoff_matrix_Game_0=[4,4]
    elif action0==0 and action1==1:        # Row player cooperates, column player defects;
        payoff_matrix_Game_0=[0,5]
    elif action0==1 and action1==0:        # Row player defects, column player cooperates;
        payoff_matrix_Game_0=[5,0]
    else:                                  # Both the agents defect;
        payoff_matrix_Game_0=[1,1]
    return(payoff_matrix_Game_0)

In [ ]:
################################################
# Payoff matrix of repeated prisoner's dilemma #
################################################

#     C    D
#  C 4,4  1,1
#  D 1,1  1,1

# action0: action of row player; 0 for cooperation, C; 1 for defection, D;
# action1: action of column player; 0 for cooperation, C; 1 for defection, D;

payoff_matrix_Game_1=np.zeros(2)
def Game_1(action0,action1):
    if action0==0 and action1==0:          # Both the agents cooperate;
        payoff_matrix_Game_1=[4,4]
    elif action0==0 and action1==1:        # Row player cooperates, column player defects;
        payoff_matrix_Game_1=[1,1]
    elif action0==1 and action1==0:        # Row player defects, column player cooperates;
        payoff_matrix_Game_1=[1,1]
    else:                                  # Both the agents defect;
        payoff_matrix_Game_1=[1,1]
    return(payoff_matrix_Game_1)

In [ ]:
###################################
# Choice of Agents under System 1 #
###################################

# memory: memory of agents:
    # memory[i][0]: memory of agent i relative to action C;
    # memory[i][1]: memory of agent i relative to action D;

def choice_s1(M,memory):                         # actions_s1=0: intuitive cooperation; actions_s1=1: intuitive defection
    N=M*2
    actions_s1= np.zeros(N)
    for i in range(N):                           # Loop inside agents
        rd.seed()
        if memory[i][0]>memory[i][1]:                # Memory of cooperation is greater than memory of defection
            actions_s1[i]=0                              # Intuitive cooperation
        elif memory[i][0]<memory[i][1]:              # Memory of defection is greater than memory of cooperation
            actions_s1[i]=1                              # Intuitive defection
        elif memory[i][0]==memory[i][1]:             # Memories of cooperation and defection are equal
            actions_s1[i]=rd.randint(0, 1)               # Random intuitive response; 50% each strategy
    return(actions_s1)

In [ ]:
###################################
# Choice of Agents under System 2 #
###################################

def choice_s2(M,Random_game,Matching):
    N=M*2
    actions_s2= np.zeros(N)
    for i in range(M):                                  # loop inside couples
        if Random_game[i]==0:                               # Couple i plays Game_0   
            actions_s2[int(Matching[i][0])]=1                   # The first agent of the couple defects under deliberation
            actions_s2[int(Matching[i][1])]=1                   # The second agent of the couple defects under deliberation
        else:                                               # Couple i plays Game_1
            actions_s2[int(Matching[i][0])]=0                   # The first agent of the couple cooperates under deliberation
            actions_s2[int(Matching[i][1])]=0                   # The second agent of the couple cooperates under deliberation
    return(actions_s2)

In [ ]:
##############################
# Choice of Cognitive System #
##############################

def system_choice(M,A,k,Matching):      # system=0: Intuition; system=1: Deliberation
    N=M*2
    system=np.zeros(N)            
    for i in range(M):                              # Loop inside couples
        if A>=rd.uniform(0, 1):                         # With probability A
            if k>=rd.uniform(0, 1):                         # With probability k
                system[int(Matching[i][0])]=0                   # First agent of the couple is intuitive
                system[int(Matching[i][1])]=0                   # Second agent of the couple is intuitive
            else:                                           # With probability 1-k
                system[int(Matching[i][0])]=1                   # First agent of the couple is deliberative
                system[int(Matching[i][1])]=1                   # Second agent of the couple is deliberative
        else:                                           # With probability 1-assortativity
            if k>=rd.uniform(0, 1):                         # With probability k
                system[int(Matching[i][0])]=0                   # First agent of the couple is intuitive
            else:                                           # With probability 1-k
                system[int(Matching[i][0])]=1                   # First agent of the couple is deliberative
            if k>=rd.uniform(0, 1):                         # With probability k
                system[int(Matching[i][1])]=0                   # Second agent of the couple is intuitive
            else:                                           # With probability 1-k
                system[int(Matching[i][1])]=1                   # Second agent of the couple is deliberative                    
    return(system)

In [ ]:
####################
# Choice of Agents #
####################

def choice(M,System,Choice_s1,Choice_s2):
    N=M*2
    actions=np.zeros(N)
    for i in range(N):                                # Loop inside agents          
        if System[i]==0:                                  # The agent is intuitive            
            actions[i]=Choice_s1[i]                           # Choice of the agent under System 1               
        else:                                             # The agent is deliberative
            actions[i]=Choice_s2[i]                           # Choice of the agent under System 2               
    return(actions)

In [ ]:
####################
# Reward of Agents #
####################

def reward(M,Random_game,Choice,Matching):
    result=np.zeros([M,2])
    for i in range(M):                                                                   #Loop inside couples
        if Random_game[i]==0:                                                                # Game_0 occurs
            result[i]=Game_0(Choice[int(Matching[i][0])],Choice[int(Matching[i][1])])            # Payoffs given the choices of the two agents forming the couple
        else:                                                                                # Game_1 occurs
            result[i]=Game_1(Choice[int(Matching[i][0])],Choice[int(Matching[i][1])])            # Payoffs given the choices of the two agents forming the couple           
    return(result)

In [ ]:
####################################
# Reward of Agents under intuition #
####################################

def reward_intuition(M,System,Reward,Matching):
    N=M*2
    reward_s1=0
    for i in range(M):                                         # Loop inside couples
        if System[int(Matching[i][0])]==0:                         # The first agent in the couple responds intuitively
            reward_s1=reward_s1+Reward[i][0]                           # Sum the reward
        if System[int(Matching[i][1])]==0:                         # The second agent in the couple responds intuitively
            reward_s1=reward_s1+Reward[i][1]                           # Sum the reward
    if sum(System)==N:                                         # All the agents deliberate
        mean_reward_s1=0                                           # Average reward under intuition equal to zero
    elif sum(System) < N:                                      # There is intuitive response
        mean_reward_s1=reward_s1/(N-sum(System))                   # Calculate the average reward
    return(mean_reward_s1)

In [ ]:
###################################
# Number of intuitive cooperators #
###################################

# Notice that agents with memories of cooperation and defection equal are counted as one half;

def cooperation(M,Choice,memory):
    N=M*2
    coop_s1=0
    for i in range(N):                      # Loop inside agents
        if memory[i][0]>memory[i][1]:           # Agent with memory of cooperation greater than memory of defection
            coop_s1=coop_s1+1                       # Sum one
        elif memory[i][0]==memory[i][1]:        # Agent with memory of cooperation equal to memery of defection
            coop_s1=coop_s1+0.5                     # Sum one half
    return(coop_s1)

In [ ]:
###################################
# Number of effective cooperators #
###################################

def cooperation_tot(M,Choice):
    N=M*2
    coop_tot=0                   
    for i in range(N):                   # Loop inside agents                    
        if Choice[i]==0:                     # The agent cooperates
            coop_tot=coop_tot+1                  # Sum one           
    return(coop_tot)

In [ ]:
##############################################
# Update Memory # Memory length equal to one #
##############################################

def update_memory_1(M,Choice,memory,Reward,vector_agents):
    N=M*2
    for i in range(M):                                                                 # Loop inside first M agents           
        if Choice[(int(vector_agents[i]))]==0:                                             # The agent cooperates
            memory[(int(vector_agents[i]))][0]=Reward[i][0]                                    # Update memory of cooperation
            memory[(int(vector_agents[i]))][1]=memory[(int(vector_agents[i]))][1]              # Memory of defection unchanged
                        
        else:                                                                              # The agent defects
            memory[(int(vector_agents[i]))][0]=memory[(int(vector_agents[i]))][0]              # Memory of cooperation unchanged
            memory[(int(vector_agents[i]))][1]=Reward[i][0]                                    # Update memory of defection
                        
    for i in range(M,N):                                                               # Loop inside second M agents
        if Choice[(int(vector_agents[i]))]==0:                                             # The agent cooperates
            memory[(int(vector_agents[i]))][0]=Reward[N-1-i][1]                                # Update memory of cooperation
            memory[(int(vector_agents[i]))][1]=memory[(int(vector_agents[i]))][1]              # Memory of defection unchanged
                        
        else:                                                                              # The agent defects
            memory[(int(vector_agents[i]))][0]=memory[(int(vector_agents[i]))][0]              # Memory of cooperation unchanged
            memory[(int(vector_agents[i]))][1]=Reward[N-1-i][1]                                # Update memory of defection
    return(memory)

In [ ]:
##########################################
# Update Memory # Reinforcement learning #
##########################################

def update_reinforce(M,Choice,memory,Reward,vector_agents):
    N=M*2
    for i in range(M):                                                                                   # Loop inside first M agents
        if Choice[(int(vector_agents[i]))]==0:                                                               # The agent cooperates
            memory[(int(vector_agents[i]))][0]=(memory[(int(vector_agents[i]))][0]+Reward[i][0])/2               # Update memory of cooperation
            memory[(int(vector_agents[i]))][1]=memory[(int(vector_agents[i]))][1]                                # Memory of defection unchanged
                       
        else:                                                                                                # The agent defects
            memory[(int(vector_agents[i]))][0]=memory[(int(vector_agents[i]))][0]                                # Memory of cooperation unchanged
            memory[(int(vector_agents[i]))][1]=(memory[(int(vector_agents[i]))][1]+Reward[i][0])/2               # Update memory of defection
                        
    for i in range(M,N):                                                                                 # Loop inside second M agents
        if Choice[(int(vector_agents[i]))]==0:                                                               # The agent cooperates
            memory[(int(vector_agents[i]))][0]=(memory[(int(vector_agents[i]))][0]+Reward[N-1-i][1])/2           # Update memory of cooperation
            memory[(int(vector_agents[i]))][1]=memory[(int(vector_agents[i]))][1]                                # Memory of defection unchanged
                        
        else:                                                                                                # The agent defects
            memory[(int(vector_agents[i]))][0]=memory[(int(vector_agents[i]))][0]                                # Memory of cooperation unchanged
            memory[(int(vector_agents[i]))][1]=(memory[(int(vector_agents[i]))][1]+Reward[N-1-i][1])/2           # Update memory of defection
                        
                  
    return(memory)

In [ ]:
######################################################################################################
# Perform simulations for different values of parameters (p, k, A), and save results in a .xlsx file #
######################################################################################################

from datetime import datetime     # Import libraries
import openpyxl
import xlsxwriter

P = np.arange(0.05, 1, 0.05)      # Define the domain of parameter p 
K = np.arange(0.05, 1, 0.05)      # Define the domain of parameter k
Assort = np.arange(0, 1.01, 0.1)  # Define the domain of parameter A


#workbook = openpyxl.load_workbook('name_of_the_file.xlsx')                                 # Open the .xlsx file                    
for a in Assort:                                                                            # Loop inside the domain of A
    globals()["S1_" + str(int(a*10))] = np.zeros([19,19])
    globals()["tot_" + str(int(a*10))] = np.zeros([19,19])
    
    #worksheet = workbook.create_sheet(r'sheet_%s' %round(a,2))                                 # Create a new sheet in the file for each value of A
    
    for p in P:                                                                                 # Loop inside the domain of p
        for k in K:                                                                                 # Loop inside the domain of k
            Simulation=simulation(100,round(p,2),round(k,2),round(a,2),25)                              # Perform simulation;   
                                                                                                        # The first argument is the number of time steps T; T=5000 in the simulations of the paper;
                                                                                                        # The last argument is the number of couples M;
                                                                                                        # The other arguments are the values of p, k, and a that follow from the loops
            
            now = datetime.now()                                                                         
            print(round(a,2),round(p,2),round(k,2),now.strftime("%H:%M:%S"))                            # Print the value of the parameters and the current time after each step of the loops
            
            globals()["S1_" + str(int(round(a,2)*10))][int(round(p,2)*20-1),int(round(k,2)*20-1)] = Simulation[0]      # For each value of A, generate a matrix with values of p on rows and values of k on columns
                                                                                                                       # Example: when A=0.8, call the matrix S1_8; 
            globals()["tot_" + str(int(round(a,2)*10))][int(round(p,2)*20-1),int(round(k,2)*20-1)] = Simulation[1]
            
            #worksheet.cell(row=int(round(p,2)*20), column=int(round(k,2)*20)).value = Simulation[0]
            #worksheet.cell(row=int(round(p,2)*20+21), column=int(round(k,2)*20)).value = Simulation[1]
    
    #workbook.save('name_of_the_file.xlsx')    # Save the file

In [ ]:
########################## 
# Graphic representation #
##########################

# Import libraries
%matplotlib notebook                  
import matplotlib
from mpl_toolkits import mplot3d
from matplotlib import cm

cmap = cm.seismic
norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)  # Scale of the color bar

X = np.arange(0.05, 1, 0.05)          # Use the same domain of k, K
Y = np.arange(0.05, 1, 0.05)          # Use the same domain of p, P

X, Y = np.meshgrid(X, Y)

fig, ax = plt.subplots()

ax.pcolormesh(X, Y, S1_10,cmap=cmap,shading='gouraud',norm=norm)     # Change the third argument to plot different results;
                                                                    
ax.set_xlabel('$K$')
ax.set_ylabel('$p$')

cax = fig.add_axes([0.905, 0.11, 0.01, 0.77])                        # Position of the colorbar
fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),cax=cax)